<a href="https://colab.research.google.com/github/Myrto-Iglezou/AI2-project4/blob/master/Question_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## YΣ19 Artificial Intelligence II
# Homework 4

### Iglezou Myrto - 111520170038


<img src="https://venturebeat.com/wp-content/uploads/2020/03/CORD-19.png?w=1200&strip=all" alt="Cord-19" width="600"/>

# Project Description



The objective of this project is about developing a document retrieval system to return titles of scientific papers containing the answer to a given user question. The dataset used in this exercise is from [COVID-19 Open Research Dataset (CORD-19)](https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases.html), the first version. We are gonna implement 2 different sentence embedding approaches, in order for the model to retrieve the titles of the papers related to a given question.


# **Question 1** 

## Step 1 - Preprocess the provided dataset

### Read all json files from folder and keep for dataset the title and the body. Then save the dataframe to a csv file for faster reading of the dataframe.

In [ ]:
import io
import os
from google.colab import drive
import pandas as pd 
import json

drive.mount('/content/drive',force_remount=True)
path = r"/content/drive/My Drive/cord-19_2020-03-13/cord-19_2020-03-13/2020-03-13/comm_use_subset"

dataset_df = pd.DataFrame(columns=['id', 'title', 'body'])

for filename in os.listdir(path):
   with open(os.path.join(path, filename), 'r') as f:  
      json_text = json.load(f)

      id = json_text['paper_id']
      # print(id)
      title = json_text['metadata']['title']
      # print(title)
      body = json_text['body_text']
      # print(body)

      dataset_df.loc[len(dataset_df)] = [id,title,body]
   

Mounted at /content/drive


In [ ]:
dataset_df.to_csv('dataset.csv',index=False)
!cp dataset.csv "drive/My Drive/"

### Read the dataset from the csv file and save the information in a dataframe

In [ ]:
import io
from google.colab import drive
import pandas as pd 
import sys 

drive.mount('/content/drive',force_remount=True)
filePath = r"/content/drive/My Drive/dataset.csv"
dataset_df = pd.read_csv(filePath)
dataset_df.fillna('empty',inplace=True)
dataset_df.title = dataset_df.title.astype(str)  # make everything str, for lower() function

Mounted at /content/drive


**Dataframe before the preprocess**

In [ ]:
dataset_df.head(5)

,id,title,body
0,236bd666a76213bc131969e1d5b66e410fc1cd45,MINI REVIEW Acute Phase Proteins in Marine Mam...,[{'text': 'The mammalian immune system include...
1,14374db205f6934d9cba148624462000bc6ec7be,Antibody Treatment against Angiopoietin-Like 4...,[{'text': 'IMPORTANCE Despite extensive global...
2,af678e8cd31d74cdb2d690addc19d59dca331f2b,Quantifying the seasonal drivers of transmissi...,"[{'text': ""Growing human population, urbanizat..."
3,42b049c2b5b32c094dc8b10f967e43ac2169b890,Evaluation of the influenza-like illness surve...,[{'text': 'the first evaluation of the Tunisia...
4,1664a9df618ca74e099245a2bd65f3172aeac284,empty,"[{'text': 'Worldwide, lung cancer remains the ..."


Remove some of the special characters, such as [, { , ' , : and some words form json like 'text'.

In [ ]:
def removeCharacters(x):
  x = x.str.replace(r'\'text\'', '')
  x = x.str.replace(r'\'start\'', '')
  x = x.str.replace(r'\'end\'', '')
  x = x.str.replace(r'[{}]', '')
  x = x.str.replace(r'[<>]', '')
  x = x.str.replace(r'[\[\]]', '')
  x = x.str.replace(r'["]', '')
  x = x.str.replace(r'[\']', '')
  x = x.str.replace(r'[:]', '')
  x = x.str.replace(r'[()]', '')
  x = x.str.replace(r'[?]', '')
  return x

In [ ]:
dataset_df['body'] = removeCharacters(dataset_df['body'])

Remove all the uppercase letters from title and body

In [ ]:
dataset_df['body'] = dataset_df['body'].apply(lambda x: x.lower())

**Dataframe after the preprocess**

In [ ]:
dataset_df.head(5)

,id,title,body
0,236bd666a76213bc131969e1d5b66e410fc1cd45,MINI REVIEW Acute Phase Proteins in Marine Mam...,the mammalian immune system includes innate o...
1,14374db205f6934d9cba148624462000bc6ec7be,Antibody Treatment against Angiopoietin-Like 4...,"importance despite extensive global efforts, ..."
2,af678e8cd31d74cdb2d690addc19d59dca331f2b,Quantifying the seasonal drivers of transmissi...,"growing human population, urbanization and gl..."
3,42b049c2b5b32c094dc8b10f967e43ac2169b890,Evaluation of the influenza-like illness surve...,the first evaluation of the tunisian influenz...
4,1664a9df618ca74e099245a2bd65f3172aeac284,empty,"worldwide, lung cancer remains the most frequ..."


Download transformers and sentence transformers

In [ ]:
%%capture
!pip install transformers

In [ ]:
%%capture
!pip install sentence-transformers

Check if GPU  is available for model to run

In [ ]:
import torch
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
    device = 'cuda'
else:
    print('No GPU available, training on CPU.')
    device = 'cpu'

No GPU available, training on CPU.


## Questions

In [ ]:
questions =  [
                  "What are the coronoviruses?",
                  "What was discovered in Wuhuan in December 2019?",
                  "What is Coronovirus Disease 2019?",
                  "What is COVID-19?",
                  "What is caused by SARS-COV2?",
                  "How is COVID-19 spread?",
                  "Where was COVID-19 discovered?",
                  "How does coronavirus spread?",
              ]

myquestions = [
                  "What are the symptoms of COVID-19?",
                  "How fast is COVID-19 spread?",
                  "What are the precautionary measures for COVID-19?",
                  "What are the symptoms of coronavirus?",
              ]

## Create the list of sentences



In [ ]:
import nltk
from nltk import tokenize
nltk.download('punkt')

ListOfBodies = dataset_df['body'].apply(lambda x: tokenize.sent_tokenize(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Usefull functions for the models

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def most_similar(sentences, embeddings, query_embedding, k = 1):
    X = np.stack(embeddings)
    score_map = zip(sentences, cosine_similarity(X, query_embedding.reshape(1, -1)))
    return sorted(score_map, key=lambda v: v[1], reverse=True)[:k]

In [ ]:
def ask_question(model,question,ListOfSentences,sentence_embeddings,dataset_df):
  query_vec = model.encode([question])[0]
  similar = most_similar(ListOfSentences,sentence_embeddings,query_vec)
  row = dataset_df[dataset_df['body'].str.contains(similar[0][0])]
  title = row['title'].tolist()[0]
  text = similar[0][0]

  return title,text

In [ ]:
from termcolor import colored

def print_answer(question,title,text):

  print(colored("Question : ",attrs=['bold']),question)
  print("\n")
  print(colored("Title : ",attrs=['bold']),title)
  print("\n")
  print(colored("Text : ",attrs=['bold']),text)
  print("\n")

## Step 2.a - First sentence embedding approach

### SBERT Model

Uses Sentence-BERT (SBERT), a modification of the BERT network using siamese and triplet networks that is able to derive semantically meaningful sentence embeddings. This allows more efficient semantic search, which is utilized in the following application.

The siamese network architecture enables that fixed-sized vectors for input sentences can be derived. Using a similarity measure like cosine similarity or Manhatten / Euclidean distance, semantically similar sentences can be found. Cosine similarity is used in this work.

SBERT fine tuned on NLI data which creates SOTA sentence embeddings, as reported in the [SBERT paper](https://arxiv.org/pdf/1908.10084.pdf).

SBERT Framework example

<img src="https://combine.se/wp-content/uploads/2019/09/3.png" alt="Cord-19" width="300"/>

Make a list of certain amount of articles

In [ ]:
ListOfSentences = []
numOfArticles = 0
for text in ListOfBodies:
  ListOfSentences += text
  numOfArticles+=1
  if(numOfArticles == 4000):
    break

In [ ]:
import torch
from sentence_transformers import SentenceTransformer

sbert_model = SentenceTransformer('bert-base-nli-mean-tokens',device=device)

100%|██████████| 405M/405M [00:15<00:00, 26.5MB/s]


In [ ]:
import time

start_time = time.time()

sentence_embeddings = sbert_model.encode(ListOfSentences,device=device)

print("-------------------------------------------------------------------------\n")
print("Time: %s seconds" % (time.time() - start_time))
print("-------------------------------------------------------------------------\n")

-------------------------------------------------------------------------

Time: 2843.639807701111 seconds
-------------------------------------------------------------------------



Check model on instructor's questions

In [ ]:
for question in questions: 
  title,text = ask_question(sbert_model,question,ListOfSentences,sentence_embeddings,dataset_df)
  print("-------------------------------------------------------------------------\n")
  print_answer(question, title, text)

-------------------------------------------------------------------------

Question :  What are the coronoviruses?


Title :  A viral metagenomic survey identifies known and novel mammalian viruses in bats from Saudi Arabia


Text :  c parechoviruses.


-------------------------------------------------------------------------

Question :  What was discovered in Wuhuan in December 2019?


Title :  Old World camels in a modern world -a balancing act between conservation and genetic improvement


Text :  the photoperiod nagy & juhasz 2019 .


-------------------------------------------------------------------------

Question :  What is Coronovirus Disease 2019?


Title :  Estimated effectiveness of symptom and risk screening to prevent the spread of COVID-19


Text :  hcov-19 has been proposed as an alternate name for the virus; jiang et al., 2020 .


-------------------------------------------------------------------------

Question :  What is COVID-19?


Title :  Alignment-free method f

Check model on my Questions

In [ ]:
for question in myquestions: 
  title,text = ask_question(sbert_model,question,ListOfSentences,sentence_embeddings,dataset_df)
  print("-------------------------------------------------------------------------\n")
  print_answer(question, title, text)

-------------------------------------------------------------------------

Question :  What are the symptoms of COVID-19?


Title :  B38-CAP is a bacteria-derived ACE2-like enzyme that suppresses hypertension and cardiac dysfunction


Text :  to examine the effects of a779 on b38-cap hypotensive action  fig.


-------------------------------------------------------------------------

Question :  How fast is COVID-19 spread?


Title :  Mechanical unfolding kinetics of the SRV-1 gag-pro mRNA pseudoknot: possible implications for −1 ribosomal frameshifting stimulation OPEN


Text :  the first step corresponds to fast and reversible transition between single-stranded and intermediate states with about 6-7 nm extension change  fig.


-------------------------------------------------------------------------

Question :  What are the precautionary measures for COVID-19?


Title :  Molecular Diagnosis of Pneumonia Using Multiplex Real-Time PCR Assay RespiFinder ® SMART 22 FAST in a Group of Mo

## Step 2.b - Second sentence embedding approach

### InferSent Model 

InferSent is a sentence embeddings method that provides semantic representations for English sentences. It is trained on natural language inference data and generalizes well to many different tasks. Just like SentenceBERT, we take a pair of sentences and encode them to generate the actual sentence embeddings. Then, extract the relations between these embeddings using:

* concatenation
* element-wise product
* absolute element-wise difference.

<img src="https://miro.medium.com/max/972/1*efWq1UrOcGy2E-34OxsBHQ.png" alt="InferSent" width="300"/>

Download InferSent encoder and Glove file

In [ ]:
! mkdir encoder
! curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl
  
! mkdir GloVe
! curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
! unzip GloVe/glove.840B.300d.zip -d GloVe/

mkdir: cannot create directory ‘encoder’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  19.1M      0  0:00:07  0:00:07 --:--:-- 23.3M
mkdir: cannot create directory ‘GloVe’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   315    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0   352    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2075M  100 2075M    0     0  2044k      0  0:17:19  0:17:19 --:--:-- 1960k
Archive:  GloVe/glove.840B.300d.zip
replace GloVe/glove.840B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
drive.mount('/content/drive',force_remount=True)
sys.path.append('/content/drive/My Drive/')
!cp -r "/content/drive/My Drive/models.py" '/content/'

Mounted at /content/drive


Make a list of certain amount of articles

In [ ]:
ListOfSentences = []
numOfArticles = 0
for text in ListOfBodies:
  ListOfSentences += text
  numOfArticles+=1
  if(numOfArticles == 10):
    break

In [ ]:
import models
from models import InferSent
import torch 

V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = '/content/GloVe/glove.840B.300d.txt'
model.set_w2v_path(W2V_PATH)

In [ ]:
if(train_on_gpu):
  model = model.cuda()

Check model on instructor's questions

In [ ]:
import time

start_time = time.time()
model.build_vocab(ListOfSentences, tokenize=True)

InferSent_embeddings = []

for sentence in ListOfSentences:
 InferSent_embeddings.append(model.encode(sentence)[0])

for question in questions: 
  title,text = ask_question(model,question,ListOfSentences,InferSent_embeddings,dataset_df)
  print("-------------------------------------------------------------------------\n")
  print_answer(question, title, text)
  
print("-------------------------------------------------------------------------\n")
print("Time: %s seconds " % (time.time() - start_time))

Found 6718(/8037) words with w2v vectors
Vocab size : 6718


/content/models.py:207: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sentences = np.array(sentences)[idx_sort]


-------------------------------------------------------------------------

Question :  What are the coronoviruses?


Title :  MINI REVIEW Acute Phase Proteins in Marine Mammals: State of Art, Perspectives and Challenges


Text :  after the initial recognition of pathogens or tissue damages by the tissue-resident macrophages, which express the pattern recognition receptors prrs, a variety of different inflammatory mediators are produced by leukocytes, endothelial cells, tissue cells or are derived from plasma proteins.


-------------------------------------------------------------------------

Question :  What was discovered in Wuhuan in December 2019?


Title :  Evaluation of the influenza-like illness surveillance system in Tunisia, 2012-2015


Text :  , cite_spans  138,  141,  7, ref_id bibref6, ref_spans , section conclusions


-------------------------------------------------------------------------

Question :  What is Coronovirus Disease 2019?


Title :  MINI REVIEW Acute Phase 

*κείμενο σε πλάγια γραφή*## Step 2.c - Third sentence embedding approach

### Doc2Vec

Doc2vec is an unsupervised algorithm to generate vectors for sentence/paragraphs/documents. The algorithm is an adaptation of word2vec which can generate vectors for words. The vectors generated can be used for tasks like finding similarity between sentences/paragraphs/documents.

<img src="https://miro.medium.com/max/972/0*x-gtU4UlO8FAsRvL." alt="InferSent" width="400"/>

Εvery paragraph is mapped to a unique vector, represented by a column in matrix D and every word is also mapped to a unique vector, represented by a column in matrix W. The paragraph vector and word vectors are averaged or concatenated to predict the next word in a context. (This is for the above figure)

Function to ask question in Doc2Vec model

In [ ]:
from nltk.tokenize import word_tokenize

def Doc2Vec_ask_question(model, question, ListOfSentences,dataset_df):
  test_doc = word_tokenize(question.lower())
  test_doc_vector = model.infer_vector(test_doc)
  similar = model.docvecs.most_similar(positive = [test_doc_vector])
  text = ListOfSentences[similar[0][0]]
  row = dataset_df[dataset_df['body'].str.contains(text)]
  title = row['title'].tolist()[0]
    
  if not title or title == 'empty':
    title = row['id'].tolist()[0]   
  else: 
    title = row['title'].tolist()[0]

  return title,text

Make a list of certain amount of articles

In [ ]:
ListOfSentences = []
numOfArticles = 0
for text in ListOfBodies:
  ListOfSentences += text
  numOfArticles+=1
  if(numOfArticles == 9000):
    break

In [ ]:
import time
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

start_time = time.time()

tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(ListOfSentences)]

model = Doc2Vec(tagged_data, vector_size = 20, window = 2, min_count = 1)

print("-------------------------------------------------------------------------\n")
print("Time: %s seconds " % (time.time() - start_time))
print("-------------------------------------------------------------------------\n")

-------------------------------------------------------------------------

Time: 1137.0766062736511 seconds 
-------------------------------------------------------------------------



Check model on instructor's questions

In [ ]:
for question in questions: 
  title,text = Doc2Vec_ask_question(model,question,ListOfSentences,dataset_df)
  print("-------------------------------------------------------------------------\n")
  print_answer(question,title,text)

-------------------------------------------------------------------------

Question :  What are the coronoviruses?


Title :  Efficacy and synergy of live-attenuated and inactivated influenza vaccines in young chickens


Text :  synergy between live-attenuated and inactivated influenza vaccines in chickens, cite_spans  1182,  1186,  32, ref_id bibref31, ref_spans  59,  76,  figs 1, 2 and 3, ref_id figref0,  292,  307,  fig 3a and 3c, ref_id figref2,  946,  953,  fig 4, ref_id figref3,  1632,  1639,  fig 4, ref_id figref3, section study 1 efficacy of live and inactivated vaccines in 1-day-old chickens,  in study 1, although the types and levels of adaptive immune responses induced in young birds appeared to depend on the type of vaccine used, neither of the two vaccines could provide full protection at 5 dpc.


-------------------------------------------------------------------------

Question :  What was discovered in Wuhuan in December 2019?


Title :  Gene Variations in Cis-Acting El

Check model on my questions

In [ ]:
for question in myquestions: 
  title,text = Doc2Vec_ask_question(model,question,ListOfSentences,dataset_df)
  print("-------------------------------------------------------------------------\n")
  print_answer(question,title,text)

-------------------------------------------------------------------------

Question :  What are the symptoms of COVID-19?


Title :  Revitalization of integrated disease surveillance and response in Sierra Leone post Ebola virus disease outbreak


Text :  improvement in idsr reporting completeness and timeliness rates was partly attributed to the support supervision., cite_spans , ref_spans , section monitoring implementation of idsr,  two data quality assessments were conducted between august 2016 and july 2017. overall, data collected through the surveillance system was of good quality and there was a modest improvement in accuracy from 95.3% in the first assessment to 97% in the second assessment.


-------------------------------------------------------------------------

Question :  How fast is COVID-19 spread?


Title :  Involvement of PRRSV NSP3 and NSP5 in the autophagy process


Text :  2a and b, the gfp-lc3 puncta accumulated in nsp3-mcherry-, nsp5-mcherry-and nsp9-mcherry-tr

## Step 3 - Comparison of three models



*   The sbert model is executed on gpu for 4000 articles, becuase for more articles the system runned out of memory and without gpu it needed more than 12 hours to run, so the colab disconnected. For the 4000 articles, trained on gpu it needed 2843 seconds.
*   The Doc2Vec model was executed on cpu for 9000 articles and runned for 1137 seconds.
* The InferSent model was running on cpu for only 10 articles for 2512 seconds. When I increased the number of articles given, the system run out of memory. The answers of the questions, as expected, do not match to the questions.
*  Beside the fact that Doc2Vec was faster and more articles were "read" the results don't seem relevant to the expected ones.

From the three models we can conclude that InferSent is the less suitable, becuase it needs a lot of time to run and  a great amount of resources. Second we can classify the Doc2Vec, since despite the fact that it runs fast for a large volume of data, its results are not at all satisfactory.

From the three models we can judge, as the most suitable sentence embedding approach, the SBERT model. From this research we can expect better results for the whole amount of articles, since we know that Doc2Vec did not do that and InferSent will take too long to achieve it. So SBERT is the most promising.


## References



*   https://www.analyticsvidhya.com/blog/2020/08/top-4-sentence-embedding-techniques-using-python/
*   https://github.com/facebookresearch/InferSent
*  https://medium.com/@mishra.thedeepak/doc2vec-simple-implementation-example-df2afbbfbad5
* https://kanoki.org/2019/03/07/sentence-similarity-in-python-using-doc2vec/

